# Write your own PyTorch training loop

![Alt text](image-6.png)

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_dataset = load_dataset("glue" , "mrpc")
checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(examples):
    return tokenizer(examples["sentence1"] , examples["sentence2"] , truncation= True) # We don't use paddint = True parameter this time.

tokenized_datasets = raw_dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

data_collator = DataCollatorWithPadding(tokenizer)

In [2]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], batch_size=8, shuffle=True, collate_fn=data_collator
)

eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, shuffle=True, collate_fn=data_collator
)

In [3]:
# We grab a batch of data for looked it.
for batch in train_dataloader:
    break

print({k: v.shape for k, v in batch.items()})

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'labels': torch.Size([8]), 'input_ids': torch.Size([8, 78]), 'token_type_ids': torch.Size([8, 78]), 'attention_mask': torch.Size([8, 78])}


In [4]:
from transformers import AutoModelForSequenceClassification

checkpoint = "bert-base-cased"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# We try on a batch which we grab.
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(0.7098, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


In [6]:
from transformers import AdamW

optimizer = AdamW(model.parameters() , lr= 5e-5)

c:\Users\Emincan\miniconda3\envs\nlp\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [8]:
loss = outputs.loss
loss.backward()
optimizer.step()

# Don't forget to zero your gradients! once your optimizer step is done!
optimizer.zero_grad()

In [9]:
from transformers import get_scheduler

num_epochs = 3
num_trainings_steps = len(train_dataloader) * num_epochs
lr_scheduler = get_scheduler(
    "linear",
    optimizer= optimizer,
    num_warmup_steps=0,
    num_training_steps= num_trainings_steps
    )

In [11]:
import torch

# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# model.to(device)
# print(device)

# If you want to use Cuda GPU, you should transfer model and optimizer to GPU.
if torch.cuda.is_available():
    model.to('cuda')
    optimizer = AdamW(model.parameters(), lr=5e-5)  # Don't forget to re define optimizer after cuda.
    device = 'cuda'

c:\Users\Emincan\miniconda3\envs\nlp\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [12]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_trainings_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/1377 [00:00<?, ?it/s]

In [13]:
from datasets import load_metric

metric = load_metric("glue" , "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

C:\Users\Emincan\AppData\Local\Temp\ipykernel_14536\2621203574.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue" , "mrpc")


{'accuracy': 0.821078431372549, 'f1': 0.8756388415672913}

In [ ]:
# All Done !